In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from clust_huge_amp import *
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="sklearn.utils.extmath")


## Aladynoulli retrospective

Here we show the ability of the model to estatime when having access to all data for signature and genomic discovery, heterogeneity characterization, and cohort similarity.

In [ ]:
%load_ext autoreload
%autoreload 2



%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import

def load_model_essentials(base_path='/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/'):
    """
    Load all essential components
    """
    print("Loading components...")
    
    # Load large matrices
    Y = torch.load(base_path + 'Y_tensor.pt')
    E = torch.load(base_path + 'E_matrix.pt')
    G = torch.load(base_path + 'G_matrix.pt')
    
    # Load other components
    essentials = torch.load(base_path + 'model_essentials.pt')
    
    print("Loaded all components successfully!")
    
    return Y, E, G, essentials

# Load and initialize model:
Y, E, G, essentials = load_model_essentials()


In [ ]:
from clust_huge_amp import *
# Subset the data
Y_100k, E_100k, G_100k, indices = subset_data(Y, E, G, start_index=0, end_index=10000)

torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Initialize model with subsetted data

del Y

In [ ]:
# Load references (signatures only, no healthy)
refs = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/reference_trajectories.pt')
signature_refs = refs['signature_refs']
# When initializing the model:


In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()



readRDS = robjects.r['readRDS']
pce_data = readRDS('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_df_prevent.rds')
pce_df = pandas2ri.rpy2py(pce_data)  # Convert to pandas DataFrame
sex=pce_df['Sex'].values

# Convert to numeric: Female=0, Male=1

pce_df['sex_numeric'] = pce_df['Sex'].map({'Female': 0, 'Male': 1}).astype(int)

sex=pce_df['sex_numeric'].values
G_with_sex = np.column_stack([G_100k, sex])  # sex should be numeric (e.g., 0/1)
# N

In [ ]:


model = AladynSurvivalFixedKernelsAvgLoss_clust_logitInit_psitest(
    N=Y_100k.shape[0], 
    D=Y_100k.shape[1], 
    T=Y_100k.shape[2], 
    K=20,
    P=G_with_sex.shape[1],
    init_sd_scaler=1e-1,
    G=G_with_sex, 
    Y=Y_100k,
    genetic_scale=1,
    W=0.0001,
    R=0,
    prevalence_t=essentials['prevalence_t'],
    signature_references=signature_refs,  # Only pass signature refs
    healthy_reference=True,  # Explicitly set to None
    disease_names=essentials['disease_names']
)

torch.manual_seed(0)
np.random.seed(0)
# Initialize with psi and clusters


import cProfile
import pstats
from pstats import SortKey

# Now in your batch run, load and verify:
initial_psi = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/initial_psi_400k.pt')
initial_clusters = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/initial_clusters_400k.pt')

model.initialize_params(true_psi=initial_psi)
model.clusters = initial_clusters
# Verify clusters match
clusters_match = np.array_equal(initial_clusters, model.clusters)
print(f"\nClusters match exactly: {clusters_match}")


# you can see that we use the full unmodified E matrix here

In [ ]:
## now we will run the model with the full data and sex specific
import cProfile
import pstats
from pstats import SortKey
# 5. Train on FULL data
profiler = cProfile.Profile()
profiler.enable()

history_new = model.fit(E_100k, num_epochs=200, learning_rate=1e-1, lambda_reg=1e-2)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats(SortKey.CUMULATIVE)
stats.print_stats(20)

In [ ]:
from utils import *
plot_training_evolution(history_new)

In [ ]:

# Save model
W=0.0001
torch.save({
    'model_state_dict': model.state_dict(),
    'phi': model.phi,
    'Y': model.Y,
    'prevalence_t': model.prevalence_t,
    'logit_prevalence_t': model.logit_prev_t,
    'G': model.G,
}, f'/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_model_W{W}_fulldata_sexspecific_921.pt')

In [ ]:

plot_theta_differences(model, diseases=[112,67,127,10,17,114], signatures=[5,7,0,17,19,5])

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Load both models
W = 0.0001  # or whatever W value you used
model_old = torch.load(f'/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/enrollment_model_W{W}_fulldata_sexspecific.pt')
model_new = torch.load(f'/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_model_W{W}_fulldata_sexspecific_921.pt')

# Extract phi and lambda parameters
phi_old = model_old['model_state_dict']['phi'].detach().numpy()
phi_new = model_new['model_state_dict']['phi'].detach().numpy()

lambda_old = model_old['model_state_dict']['lambda_'].detach().numpy()
lambda_new = model_new['model_state_dict']['lambda_'].detach().numpy()

# Compare phi parameters
print("PHI COMPARISON:")
print(f"Phi shape: {phi_old.shape}")
print(f"Phi difference (L2 norm): {np.linalg.norm(phi_new - phi_old):.6f}")
print(f"Phi relative difference: {np.linalg.norm(phi_new - phi_old) / np.linalg.norm(phi_old):.6f}")
print(f"Phi max absolute difference: {np.max(np.abs(phi_new - phi_old)):.6f}")

# Compare lambda parameters
print("\nLAMBDA COMPARISON:")
print(f"Lambda shape: {lambda_old.shape}")
print(f"Lambda difference (L2 norm): {np.linalg.norm(lambda_new - lambda_old):.6f}")
print(f"Lambda relative difference: {np.linalg.norm(lambda_new - lambda_old) / np.linalg.norm(lambda_old):.6f}")
print(f"Lambda max absolute difference: {np.max(np.abs(lambda_new - lambda_old)):.6f}")


In [ ]:

# Plot comparisons
# Fixed plotting code
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Phi comparison - scatter plot
axes[0, 0].scatter(phi_old.flatten(), phi_new.flatten(), alpha=0.5)
axes[0, 0].plot([phi_old.min(), phi_old.max()], [phi_old.min(), phi_old.max()], 'r--')
axes[0, 0].set_xlabel('Old Phi')
axes[0, 0].set_ylabel('New Phi')
axes[0, 0].set_title('Phi: Old vs New')

# Lambda comparison - scatter plot
axes[0, 1].scatter(lambda_old.flatten(), lambda_new.flatten(), alpha=0.5)
axes[0, 1].plot([lambda_old.min(), lambda_old.max()], [lambda_old.min(), lambda_old.max()], 'r--')
axes[0, 1].set_xlabel('Old Lambda')
axes[0, 1].set_ylabel('New Lambda')
axes[0, 1].set_title('Lambda: Old vs New')

# Phi difference heatmap - average across diseases
phi_diff = phi_new - phi_old
phi_diff_avg = np.mean(phi_diff, axis=1)  # Average across diseases
im1 = axes[1, 0].imshow(phi_diff_avg, aspect='auto', cmap='RdBu_r')
axes[1, 0].set_title('Phi Difference (New - Old) - Avg across diseases')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Signature')
plt.colorbar(im1, ax=axes[1, 0])

# Lambda difference heatmap - first few individuals, first signature
lambda_diff = lambda_new - lambda_old
im2 = axes[1, 1].imshow(lambda_diff[:10, 0, :], aspect='auto', cmap='RdBu_r')
axes[1, 1].set_title('Lambda Difference (New - Old) - First 10 individuals, Signature 0')
axes[1, 1].set_xlabel('Time')
axes[1, 1].set_ylabel('Individual')
plt.colorbar(im2, ax=axes[1, 1])

plt.tight_layout()
plt.show()

print(f"\nSUMMARY:")
print(f"Models are IDENTICAL - no changes detected!")
print(f"Phi correlation: {np.corrcoef(phi_old.flatten(), phi_new.flatten())[0,1]:.4f}")
print(f"Lambda correlation: {np.corrcoef(lambda_old.flatten(), lambda_new.flatten())[0,1]:.4f}")

In [ ]:
model_new['model_state_dict'].keys()